In [1]:
import pandas as pd
import numpy as np

In [2]:
input_file = pd.read_csv('home_ac/processed_hhdata_86_2.csv')[0:100][['use','AC']]
input_file.head(10)

,use,AC
0,0.500100,0.000000
1,0.544467,0.000000
2,0.501350,0.000000
3,0.464817,0.000000
4,0.473133,0.000000
5,0.467667,0.000000
6,0.526833,0.000000
7,0.971033,0.000000
8,1.106833,0.000000
9,1.777617,0.083074


In [3]:
input_result = pd.read_csv('model_result/86_2/sb20b64.csv')[0:100][['Base_Soc','Base_Action']]
input_result.head(10)

,Base_Soc,Base_Action
0,2.999360,0.000000
1,2.998720,0.000000
2,2.998080,0.000000
3,2.997440,0.000000
4,2.996800,0.000000
5,2.996160,0.000000
6,4.895520,2.000000
7,2.789617,-2.000000
8,0.683714,-2.000000
9,0.000000,-0.683714


In [4]:
pair= pd.concat([input_file,input_result],axis=1)
pair.head(20)

,use,AC,Base_Soc,Base_Action
0,0.500100,0.000000,2.999360,0.000000
1,0.544467,0.000000,2.998720,0.000000
2,0.501350,0.000000,2.998080,0.000000
3,0.464817,0.000000,2.997440,0.000000
4,0.473133,0.000000,2.996800,0.000000
5,0.467667,0.000000,2.996160,0.000000
6,0.526833,0.000000,4.895520,2.000000
7,0.971033,0.000000,2.789617,-2.000000
8,1.106833,0.000000,0.683714,-2.000000
9,1.777617,0.083074,0.000000,-0.683714


In [5]:
#sample 1000 from training set
#The Matr
sample=pair.sample(20)
#print(sample)
con1=sample.cov()
print(con1)

                  use        AC  Base_Soc  Base_Action
use          1.916170  0.168566 -0.108662    -0.100465
AC           0.168566  0.634099 -0.031434    -0.073671
Base_Soc    -0.108662 -0.031434  0.180378     0.197365
Base_Action -0.100465 -0.073671  0.197365     0.326095


In [6]:
def kernal(m1,m2):
    global con1
    result= np.exp ((-(np.subtract(m1,m2))*np.linalg.inv(con1/2)*(np.subtract(m1,m2)).transpose()).item((0,0)))
    return result

In [7]:
all_states=[]
for i in range(len(pair)):
    feature_v=[]
    for j in range(len(sample)):
        step_m = np.matrix([[pair['use'][i],pair['AC'][i],pair['Base_Soc'][i],pair['Base_Action'][i]]])
        sample_m = np.matrix([[sample['use'].iloc[j],sample['AC'].iloc[j],sample['Base_Soc'].iloc[j],sample['Base_Action'].iloc[j]]])
        feature_v.append((kernal(step_m,sample_m)))
    all_states.append(feature_v)
all_states

[[1.5159534599327527e-134,
  4.54510977291587e-130,
  2.463408209867203e-131,
  1.3436441860854793e-89,
  5.6393210588353163e-154,
  3.0323567046124956e-130,
  3.0910739652212177e-131,
  1.0650118002232038e-129,
  2.2953397681836303e-132,
  2.8636857494262704e-132,
  2.874964823094126e-132,
  1.3915583438935876e-130,
  1.3932520637083784e-71,
  5.975275169031082e-131,
  2.2788246792540653e-131,
  2.2126000749105863e-135,
  2.4184875830937027e-132,
  1.4124313566314386e-131,
  8.688834307480583e-132,
  1.1494354473955347e-130],
 [1.4552352703377763e-134,
  4.664394588931711e-130,
  2.3170432274065567e-131,
  1.3012824330602569e-89,
  5.1187545621403586e-154,
  3.062143840791501e-130,
  2.9227745971367196e-131,
  1.267729451270136e-129,
  2.0576048575082817e-132,
  2.5775203816027996e-132,
  2.5878605505711283e-132,
  1.369256550660707e-130,
  1.337435373559467e-71,
  5.742532854954838e-131,
  3.106817772812752e-131,
  3.117089826204854e-135,
  2.1700626542419353e-132,
  1.31224390033448

In [8]:
Feature_reg=pd.DataFrame(all_states)
Feature_reg

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,1.515953e-134,4.545110e-130,2.463408e-131,1.343644e-89,5.639321e-154,3.032357e-130,3.091074e-131,1.065012e-129,2.295340e-132,2.863686e-132,2.874965e-132,1.391558e-130,1.393252e-71,5.975275e-131,2.278825e-131,2.212600e-135,2.418488e-132,1.412431e-131,8.688834e-132,1.149435e-130
1,1.455235e-134,4.664395e-130,2.317043e-131,1.301282e-89,5.118755e-154,3.062144e-130,2.922775e-131,1.267729e-129,2.057605e-132,2.577520e-132,2.587861e-132,1.369257e-130,1.337435e-71,5.742533e-131,3.106818e-131,3.117090e-135,2.170063e-132,1.312244e-131,7.990865e-132,1.124627e-130
2,1.955151e-134,5.852248e-130,3.170766e-131,1.633454e-89,7.301192e-154,3.904189e-130,3.978747e-131,1.372110e-129,2.953872e-132,3.685336e-132,3.699853e-132,1.791460e-130,1.664041e-71,7.691710e-131,2.937625e-131,2.857810e-135,3.112363e-132,1.817916e-131,1.118280e-131,1.479723e-130
3,2.552643e-134,7.208283e-130,4.204265e-131,2.004078e-89,1.003231e-153,4.874896e-130,5.252147e-131,1.490766e-129,4.079253e-132,5.071961e-132,5.091626e-132,2.286479e-130,2.023613e-71,1.001480e-130,2.845677e-131,2.697156e-135,4.294667e-132,2.435709e-131,1.511250e-131,1.897670e-130
4,2.816952e-134,8.043150e-130,4.619194e-131,2.161724e-89,1.098007e-153,5.423958e-130,5.775892e-131,1.707802e-129,4.443758e-132,5.529150e-132,5.550659e-132,2.532324e-130,2.163560e-71,1.104530e-130,3.339488e-131,3.186052e-135,4.679211e-132,2.670249e-131,1.653785e-131,2.099600e-130
5,3.272194e-134,9.247802e-130,5.374826e-131,2.424942e-89,1.288257e-153,6.250121e-130,6.715893e-131,1.924085e-129,5.204866e-132,6.472560e-132,6.497675e-132,2.928427e-130,2.405614e-71,1.281430e-130,3.693195e-131,3.511343e-135,5.479931e-132,3.112312e-131,1.930257e-131,2.429900e-130
6,1.314790e-211,2.637256e-207,1.590028e-209,4.065302e-132,1.006877e-229,1.171862e-207,2.278401e-209,2.594810e-205,4.411216e-211,6.095726e-211,6.130959e-211,2.797986e-208,3.974081e-147,6.633643e-209,1.766634e-205,2.828689e-208,4.760708e-211,6.683822e-210,3.182598e-210,2.003832e-208
7,1.406260e-250,1.522594e-243,1.646323e-245,1.562196e-205,9.306498e-275,7.538024e-244,2.277225e-245,5.543822e-242,6.304016e-247,8.477695e-247,8.522541e-247,2.141410e-244,2.315237e-144,5.945885e-245,1.331276e-242,2.872840e-248,6.760206e-247,7.516099e-246,3.831216e-246,1.592934e-244
8,3.648662e-68,1.346120e-63,3.751246e-64,4.356271e-66,1.731851e-88,1.215976e-63,4.263473e-64,1.943982e-64,8.625501e-65,9.971646e-65,9.997253e-65,9.082583e-64,1.448120e-17,6.072830e-64,3.158269e-67,4.140546e-72,8.927193e-65,2.710943e-64,2.018608e-64,8.344396e-64
9,9.903153e-07,1.882183e-04,8.874407e-05,2.555941e-21,2.062381e-22,1.874008e-04,9.770610e-05,1.110676e-05,2.727366e-05,3.076780e-05,3.083328e-05,1.636831e-04,3.900346e-06,1.261697e-04,8.757717e-09,9.498511e-12,2.806595e-05,6.908225e-05,5.469345e-05,1.556067e-04
